In [3]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
#%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil_project/utils/')

#
import numpy as np
from tqdm import trange

#
from visualize import Visualize


#
import sleap

#
import cv2
import os

#
from tqdm import tqdm
import parmap

#
import pandas as pd

#
from track import make_human_label_only_slp, make_human_label_only_video

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [13]:
################################################################
########### SET FNAMES FOR HUMAN AND ID SWITCH FILES ###########
################################################################
from track import DatabaseLoader

#
root_dir = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/'
fname_database = 'database.xlsx'
human_slp_dir = 'human_labels'
input_dir = 'input_dir'
n_cores = 16
network_type = "Night"   # "Day" or "Night"

# load database class
d = DatabaseLoader(root_dir + fname_database,
                   network_type)
d.root_dir = root_dir
d.human_slp_dir = human_slp_dir
d.input_dir = input_dir
d.n_cores = n_cores
d.network_type = network_type

# make id switch files over all files in /input_dir/ and database.xlsx 
d.make_id_switch_files()

# process the human labeled .slp file to short version
d.fname_slp_human_labels = 'cohort2_night.1000.slp'
d.clean_human_slp()

# merge the id switch and human files

d.subsample=10
d.merge_human_and_id_switches()


print ("DONE...")


reading excel spreasheet...
 parsing dataframe of size:  1048575

running .slp preprocessing and id_switch detection ...


  0%|          | 0/468 [00:00<?, ?it/s]


making shortened id_switch .slp files...


  0%|          | 0/468 [00:00<?, ?it/s]


making shortened id_switch video file...
    movie already generated... exiting
    hybrid file already exists...exiting
   video already exists ... exiting
processing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/human_labels.avi
processing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/id_switches_cropped.mp4
done...
slp ctrl:  0
slp ctrl:  1
slp ctrl:  10
slp ctrl:  11
slp ctrl:  20
slp ctrl:  21
slp ctrl:  30
slp ctrl:  31
slp ctrl:  40
slp ctrl:  41
slp ctrl:  50
slp ctrl:  51
slp ctrl:  60
slp ctrl:  61
slp ctrl:  70
slp ctrl:  71
slp ctrl:  80
slp ctrl:  81
slp ctrl:  90
slp ctrl:  91
slp ctrl:  100
slp ctrl:  101
slp ctrl:  110
slp ctrl:  111
slp ctrl:  120
slp ctrl:  121
slp ctrl:  130
slp ctrl:  131
slp ctrl:  140
slp ctrl:  141
slp ctrl:  150
slp ctrl:  151
slp ctrl:  160
slp ctrl:  161
slp ctrl:  170
slp ctrl:  171
slp ctrl:  180
slp ctrl:  181
slp ctrl:  190
slp ctrl:  191
slp ctrl:  200
slp ctrl:  201
slp ctrl:  210
slp ctrl:  211
slp ctrl:  220
slp ctrl: 